# Using the Zarr Tile Sink

The `ZarrFileTileSource` class has file-writing capabilities; a user may start with an empty Zarr store, add data tiles, then save the data to a file of any format. 

Typically, this class is called a "source" when reading from a file and a "sink" when writing to a file. This is just a naming convention, but the read mode and write mode are not mutually exclusive.

## Installation

In [ ]:
# This will install large_image with the zarr source
!pip install large_image[tiff,zarr] --find-links https://girder.github.io/large_image_wheels --upgrade

# For maximum capabilities in Jupyter, also install ipyleaflet so you can
# view zoomable images in the notebook
!pip install ipyleaflet

## Sample Data Download

For this example, we will use data from a sample file. We will copy and modify tiles from this image, writing the modified data to a new file.

In [ ]:
!curl -L -C - -o example.tiff https://demo.kitware.com/histomicstk/api/v1/item/58b480ba92ca9a000b08c899/download

In [ ]:
import large_image

original_image_path = 'example.tiff'
processed_image_path = 'processed_example_1.tiff'

source = large_image.open(original_image_path)

# view the metadata
source_metadata = source.getMetadata()
source_metadata

In [ ]:
# show the source image in an interactive viewer
source

## Writing Processed Data to a New File

In [ ]:
from skimage.color.adapt_rgb import adapt_rgb, hsv_value
from skimage.morphology import disk
from skimage import filters

# define some image processing function

@adapt_rgb(hsv_value)
def process_tile(tile, footprint_size):
    return filters.unsharp_mask(
        tile, radius=footprint_size
    )

In [ ]:
# create a sink, which is an instance of ZarrFileTileSource and has no data
sink = large_image.new()

# compare three different footprint sizes for processing algorithm
# computing the processed image takes about 1 minute for each value
footprint_sizes = [1, 10, 50]
print(f'Processing image for {len(footprint_sizes)} frames.')

# create a frame for each processed result
for i, footprint_size in enumerate(footprint_sizes):
    print('Processing image with footprint_size = ', footprint_size)
    # iterate through tiles, getting numpy arrays for each tile
    for tile in source.tileIterator(format='numpy'):
            # for each tile, run some processing algorithm
            t = tile['tile']
            processed_tile = process_tile(t, footprint_size) * 255

            # add modified tile to sink
            # specify tile x, tile y, and any arbitrary frame parameters
            sink.addTile(processed_tile, x=tile['x'], y=tile['y'], i=i)
# view metadata
sink.getMetadata()     

In [ ]:
# show the result image in an interactive viewer
# the viewer includes a slider for this multiframe image
# switch between frames to view the differences between the values passed to footprint_size
sink

## Edit Attributes and Write Result File

In [ ]:
# set crop bounds
sink.crop = (3000, 5000, 2048, 2048)

# set mm_x and mm_y from source metadata
sink.mm_x = source_metadata.get('mm_x')
sink.mm_y = source_metadata.get('mm_y')

# set image description
sink.imageDescription = 'processed with scikit-image'

# add original thumbnail as an associated image
sink.addAssociatedImage(source.getThumbnail()[0], imageKey='original')

# write new image as tiff (other format options include .zip, .zarr, .db, .sqlite, .svs, etc.)
sink.write(processed_image_path)

## View Results

In [ ]:
# open written file as a new source
# this will be opened as a TiffFileTileSource
source_2 = large_image.open(processed_image_path)

# view metadata
source_2.getMetadata()

In [ ]:
# show source_2 in an interactive viewer
source_2